In [1]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
###############################
####     Données 2023       ###
###############################

################################
##   juste MEAN / STD         ##
################################


import os
directory = 'data/v_config1-lcb'#group3/config_1'#/config_3'   # change here to get more data
labels=[]
all_data=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    data = pd.read_csv(f)

    u = data.drop(columns=['t']).describe().T[['mean','std']].T
    number = pd.DataFrame(pd.concat([u.iloc[0], u.iloc[1]], axis=0)).T
    number['label'] = int(filename[10])
    number['iteration'] = int(filename.split('_')[3].split('.')[0])

    all_data.append(number)

to_test = pd.concat(all_data)

# merege
data1 = pd.read_csv('data/df_nb_sign_changes_v.csv')
data1 = data1[['label','iteration','nb_sign_changes_acceleration_x', 'nb_sign_changes_acceleration_y', 'nb_sign_changes_acceleration_z']]

data2 = pd.read_csv('data/csv_benoit.csv')
data2.rename(columns={"digit": "label"}, inplace=True)
data2 = data2[['label','iteration','changement_acceleration_x',
       'changement_acceleration_y', 'changement_acceleration_z',
       'changement_rotation_x', 'changement_rotation_y',
       'changement_rotation_z', 'changement_magnetisme_x',
       'changement_magnetisme_y', 'changement_magnetisme_z']]
data2['label'] = data2['label'].astype(int)


to_test1 = to_test.merge(data1, on=['label','iteration'], how='inner')
to_test3 = to_test1.merge(data2, left_index=True, right_index=True)
to_test3.drop(columns=['label_y', 'iteration_y','iteration_x'], inplace=True)
to_test3.rename(columns={"label_x": "label"}, inplace=True)

to_test.drop(columns=['iteration'], inplace=True)

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedShuffleSplit


def cross_validate_models(model,data,target):
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
    cv_results = cross_validate(model, data, target, cv=sss)
    cv_results = pd.DataFrame(cv_results)
    return cv_results




In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


models = [DecisionTreeClassifier(), LogisticRegression(max_iter=10000),SVC(), RandomForestClassifier()]  


print(50*'*')
print('juste "mean" et "sdt" du describe')
for model in models:
    cv_results = cross_validate_models(model,to_test.drop(columns='label'), to_test.label)
    print(
    "Generalization score without hyperparameters"
    f" tuning:\n{cv_results['test_score'].mean():.3f} ±"
    f" {cv_results['test_score'].std():.3f}"
)

**************************************************
DATA du describe
Generalization score without hyperparameters tuning:
0.655 ± DecisionTreeClassifier() : 0.035
Generalization score without hyperparameters tuning:
0.786 ± LogisticRegression(max_iter=10000) : 0.031
Generalization score without hyperparameters tuning:
0.267 ± SVC() : 0.051
Generalization score without hyperparameters tuning:
0.888 ± RandomForestClassifier() : 0.021


In [15]:
print(50*'*')
print('juste "mean" et "sdt" du describe + feature engineering "nb sign"')
for model in models:
    cv_results = cross_validate_models(model,to_test1.drop(columns='label'), to_test1.label)
    print(
    "Generalization score without hyperparameters"
    f" tuning:\n{cv_results['test_score'].mean():.3f} ±"
    f" {cv_results['test_score'].std():.3f}"
)

**************************************************
DATA du describe + feature engineering "nb sign"
Generalization score without hyperparameters tuning:
0.642 ± 0.045
Generalization score without hyperparameters tuning:
0.766 ± 0.034
Generalization score without hyperparameters tuning:
0.253 ± 0.043
Generalization score without hyperparameters tuning:
0.893 ± 0.018


In [16]:
print(50*'*')
print('juste "mean" et "sdt" du describe + feature engineering "nb sign" & "changement"')
for model in models:
    cv_results = cross_validate_models(model,to_test3.drop(columns='label'), to_test3.label)
    print(
    "Generalization score without hyperparameters"
    f" tuning:\n{cv_results['test_score'].mean():.3f} ±"
    f" {cv_results['test_score'].std():.3f}"
)

**************************************************
DATA du describe + feature engineering "changement"
Generalization score without hyperparameters tuning:
0.652 ± 0.041
Generalization score without hyperparameters tuning:
0.707 ± 0.054
Generalization score without hyperparameters tuning:
0.243 ± 0.043
Generalization score without hyperparameters tuning:
0.887 ± 0.036


In [17]:
def linearize(u):
    all=[]
    for line in range(len(u)):
        all.append(u.iloc[line])
    return pd.concat(all, axis=0).T

directory = 'data/v_config1-lcb'
labels=[]
all_data=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    data = pd.read_csv(f)
    u = data.drop(columns='t').describe().T.drop(['count'], axis=1)
    number = pd.DataFrame(linearize(u)).T
    number['label'] = int(filename[10])
    number['iteration'] = int(filename.split('_')[3].split('.')[0])

    all_data.append(number)
    
to_test = pd.concat(all_data)

# merege
data1 = pd.read_csv('data/df_nb_sign_changes_v.csv')
data1 = data1[['label','iteration','nb_sign_changes_acceleration_x', 'nb_sign_changes_acceleration_y', 'nb_sign_changes_acceleration_z']]
to_test2 = to_test.merge(data1, on=['label','iteration'], how='inner')


to_test.drop(columns=['iteration'], inplace=True)
to_test2.drop(columns=['iteration'], inplace=True)


In [19]:
print(50*'*')
print('all DATA du describe')
for model in models:
    cv_results = cross_validate_models(model,to_test.drop(columns='label'), to_test.label)
    print(
    "Generalization score without hyperparameters"
    f" tuning:\n{cv_results['test_score'].mean():.3f} ±"
    f" {cv_results['test_score'].std():.3f}"
)

**************************************************
all DATA du describe
Generalization score without hyperparameters tuning:
0.685 ± 0.041
Generalization score without hyperparameters tuning:
0.863 ± 0.021
Generalization score without hyperparameters tuning:
0.327 ± 0.058
Generalization score without hyperparameters tuning:
0.926 ± 0.025


In [20]:
print(50*'*')
print('all DATA du describe + "nb sign"')
for model in models:
    cv_results = cross_validate_models(model,to_test2.drop(columns='label'), to_test2.label)
    print(
    "Generalization score without hyperparameters"
    f" tuning:\n{cv_results['test_score'].mean():.3f} ±"
    f" {cv_results['test_score'].std():.3f}"
)

**************************************************
all DATA du describe + "nb sign"
Generalization score without hyperparameters tuning:
0.675 ± 0.046
Generalization score without hyperparameters tuning:
0.869 ± 0.025
Generalization score without hyperparameters tuning:
0.325 ± 0.054
Generalization score without hyperparameters tuning:
0.937 ± 0.015


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(to_test.drop("label", axis=1), to_test["label"], test_size=0.3, random_state=42, stratify=to_test["label"])

# Créer un modèle d'arbre de décision
model = DecisionTreeClassifier()